In [1]:
import modern_robotics as mr
import numpy as np
import numpy.matlib
from calibration import *
numpy.set_printoptions(precision=16)

In [2]:
w1 = np.array([0,0,1])
p1 = np.array([0,0,0])

w2 = np.array([0,0,1])
p2 = np.array([0.250,0,0])

w3 = np.array([0,0,0])
p3 = np.array([0,0,-1])

w4 = np.array([0,0,-1])
p4 = np.array([0.470,0,0])



s = 0.001
ax1 = mr.ScrewToAxis(p1,w1,0)
ax2 = mr.ScrewToAxis(p2,w2,0)
ax3 = np.r_[[0,0,0],p3]
ax4 = mr.ScrewToAxis(p4,w4,0)
xi0 = np.c_[ax1,ax2,ax3,ax4]


deq1=0
deq2=0.02
deq3=0.002
deq4=0.02

xi01= np.array([0.0199900035972015,0,0.999800179914059,0,0.0130330000000000*1000*s,0])
xi02=np.array([0,0.000399999968000004,0.999999920000010,-0.000300000000000000*1000*s,-0.253990000161600*1000*s,0.000101596000064640*1000*s])
xi03=np.array([0,0,0,0.0199999568791395,0.0195999577415567,-0.999607844797830])
xi04=np.array([0.0407700195329210,0.0391700187663605,-0.998400478333784,-0.0266829837144079*1000*s,0.504558015646471*1000*s,0.0187056011887379*1000*s])
xi00=np.c_[xi01,xi02,xi03,xi04]
N=10 

vtheta=np.c_[np.random.rand(N,1)*2*np.pi,np.random.rand(N,1)*2*np.pi,np.random.rand(N,1)*1000*s,np.random.rand(N,1)*2*np.pi]
P00=np.array([-100,-100,-100])*s
P01=np.array([100,0,0])*s
P02=np.array([0,100,0])*s
P03=np.array([0,0,100])*s
PX=np.c_[P01,P02,P03]
gm=np.zeros((4,4,N))
gn=np.zeros((4,4,N))
Pa1=np.zeros((4,N))
Pa2=np.zeros((4,N))
Pa3=np.zeros((4,N))
M_home = np.eye(4)


for i in range(0,10):
    
    thetalist = np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])

    Pa1[:,i] = np.dot(mr.FKinBody(M_home, xi00, np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])), np.r_[P01,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    Pa2[:,i] = np.dot(mr.FKinBody(M_home, xi00, np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])), np.r_[P02,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    Pa3[:,i] = np.dot(mr.FKinBody(M_home, xi00, np.array([ vtheta[i,0] + deq1, vtheta[i,1] + deq2, vtheta[i,2] + deq3, vtheta[i,3] + deq4 ])), np.r_[P03,1].reshape((4,1)) ).reshape((4,)) + ((np.random.rand(4,1)*0.2-0.1) * s).reshape((4,))
    PY = np.c_[Pa1[0:3,i],Pa2[0:3,i],Pa3[0:3,i]]

    R,t = Registration(PX, PY)

    gm[0:3,0:3,i] = R
    gm[0:3,3,i] = t
    gn[:,:,i] = mr.FKinBody(M_home, xi00, thetalist)

# calibration
xiTrad, dqTrad, meanETrad, convergenceTrad = traditionalCalibrationScara(xi0, vtheta, gm, 10)

Nt=50
error_before=np.zeros((Nt,1))
error_afterMinimal=np.zeros((Nt,1))
error_afterTraditional=np.zeros((Nt,1))
test_joint_config = np.c_[np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 2 * np.pi, np.random.rand(Nt,1) * 1000 * s, np.random.rand(Nt,1) * 2 * np.pi ]


for i in range(0,Nt):
    
    error_before[i] = np.linalg.norm( np.dot( mr.FKinBody(M_home, np.round(xi0,4), test_joint_config[i,:]) - mr.FKinBody(M_home, np.round(xi00,4), test_joint_config[i,:] + np.array([deq1,deq2,deq3,deq4]) ), np.r_[P00,1])) / s
    
    error_afterTraditional[i] = np.linalg.norm( np.dot( mr.FKinBody(M_home, np.round(xiTrad,4), (test_joint_config[i,:] + dqTrad.T).reshape( (4,) ) ) - mr.FKinBody(M_home, np.round(xi00,4), test_joint_config[i,:] + np.array([deq1,deq2,deq3,deq4]) ), np.r_[P00,1]) ) / s


print("Mean error = {}".format(np.mean(error_before)))
print("Mean error calibrated = {}".format(np.mean(error_afterTraditional)))

print("Max error before = {}".format(np.max(error_before)))
print("Max error calibrated = {}".format(np.max(error_afterTraditional)))

Mean error = 84.88577410701458
Mean error calibrated = 0.29318558208139683
Max error before = 162.90127884790965
Max error calibrated = 0.5256773078506304
